## **5. Evaluación de Modelos**


---


In [ ]:
import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import date
from joblib import dump, load

import warnings; warnings.filterwarnings("ignore")

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

import xgboost as xgb

In [ ]:
df = pd.read_csv('traffic_site_2.csv')

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error


from sklearn.model_selection import train_test_split

import xgboost as xgb

In [ ]:
def get_scores(model_name, y_test, predictions):
  r2 = r2_score(y_test, predictions)
  rmse = np.sqrt(mean_squared_error(y_test, predictions))
  return {
    'model': model_name,
    'r2': r2,
    'rmse': rmse
  }

def save_scores(model_scores, df_scores):
  scores = df_scores.copy()
  scores = pd.concat([scores, pd.DataFrame(model_scores, index=[0])], ignore_index=True)
  return scores

In [ ]:
x = df_transformado.drop('transactionRevenue',axis=1)
y = df_transformado['transactionRevenue']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state = 123)

### 4.1. XGBoost

In [ ]:
xgbr = xgb.XGBRegressor(
  eval_metric='rmse',
  random_state=123
)

In [ ]:
xgbr.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=123, ...)

In [ ]:
xgbr_predictions = xgbr.predict(x_test)

In [ ]:
xbr_scores = get_scores('XGBoost Regressor', xgbr_predictions, y_test)
xbr_scores

{'model': 'XGBoost Regressor',
 'r2': -0.5945430911182854,
 'rmse': 2.0043945339451303}

In [ ]:
scores = save_scores(xbr_scores, scores)
scores

,model,r2,rmse
0,XGBoost Regressor,0.098200,2.004395
1,xgbr,-0.594543,2.004395
2,XGBoost Regressor,-0.594543,2.004395


### 4.2. LightGBM

In [ ]:
import lightgbm as lgbm

In [ ]:
lgbmr = lgbm.LGBMRegressor(random_state=123)
lgbmr.fit(
  x_train,
  y_train
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 408
[LightGBM] [Info] Number of data points in the train set: 10440, number of used features: 102
[LightGBM] [Info] Start training from score 0.233929


LGBMRegressor(random_state=123)

In [ ]:
lgbmr_predictions = lgbmr.predict(x_test)
# lgbmr_predictions = grid.predict(x_test)

In [ ]:
model_scores = get_scores('Lightgbm', y_test, lgbmr_predictions)
model_scores

{'model': 'Lightgbm', 'r2': 0.23861611165407282, 'rmse': 1.8417464907978418}

In [ ]:
scores = pd.concat([scores, pd.DataFrame(model_scores, index=[0])], ignore_index=True)
scores

,model,r2,rmse
0,XGBoost Regressor,0.098200,2.004395
1,xgbr,-0.594543,2.004395
2,XGBoost Regressor,-0.594543,2.004395
3,Lightgbm,0.238616,1.841746


### 4.3. LinearRgression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)

LinearRegression()

In [ ]:
lr_predictions = lr.predict(x_test)

In [ ]:
lr_scores = get_scores('LinearRegression', lr_predictions, y_test)
lr_scores

{'model': 'LinearRegression',
 'r2': -0.00064067843432003,
 'rmse': 1347646957.1257772}

In [ ]:
scores = save_scores(lr_scores, scores)
scores

,model,r2,rmse
0,XGBoost Regressor,0.098200,2.004395e+00
1,xgbr,-0.594543,2.004395e+00
2,XGBoost Regressor,-0.594543,2.004395e+00
3,Lightgbm,0.238616,1.841746e+00
4,LinearRegression,-0.000641,1.347647e+09


### 4.4. RandomForest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(random_state=123)

In [ ]:
rf.fit(x_train, y_train)

RandomForestRegressor(random_state=123)

In [ ]:
rf_predictions = rf.predict(x_test)

In [ ]:
rf_scores = get_scores('RandomForest', y_test, rf_predictions)
rf_scores

{'model': 'RandomForest',
 'r2': 0.22190723610748586,
 'rmse': 1.8618457512601094}

In [ ]:
scores = save_scores(rf_scores, scores)
scores

,model,r2,rmse
0,XGBoost Regressor,0.098200,2.004395e+00
1,xgbr,-0.594543,2.004395e+00
2,XGBoost Regressor,-0.594543,2.004395e+00
3,Lightgbm,0.238616,1.841746e+00
4,LinearRegression,-0.000641,1.347647e+09
5,RandomForest,0.221907,1.861846e+00


In [ ]:
# #Mejorando el modelo de predicción
# from sklearn.model_selection import KFold
# kfolds = KFold(n_splits=5, shuffle=True)
# # Definir los parámetros que deseas ajustar

# parametros = {
#   'criterion': ['absolute_error', 'friedman_mse', 'squared_error', 'poisson'],
#   'max_depth': [10, 20, 30],
#   'max_features': ['sqrt', 'log2', None],
#   'bootstrap': [True, False],
#   'n_estimators': [50, 100, 150],
# }

# from sklearn.model_selection import RandomizedSearchCV
# rf_randomizedSearchCV = RandomizedSearchCV(
#   rf,
#   parametros,
#   n_iter=10,
#   cv=kfolds,
#   scoring='r2',
#   n_jobs=-1
# )

# # Ajustar el modelo a los datos de entrenamiento
# rf_randomized_fit = rf_randomizedSearchCV.fit(x_train, y_train)
# rf_randomized_fit.best_params_

In [ ]:
{'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 30,
 'criterion': 'squared_error',
 'bootstrap': True}

{'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 30,
 'criterion': 'squared_error',
 'bootstrap': True}

In [ ]:
rf_optimized = RandomForestRegressor(
  random_state = 123,
  n_estimators = 100,
  max_features = 'sqrt',
  max_depth = 30,
  criterion = 'squared_error',
  bootstrap = True
)
rf_optimized.fit(x_train, y_train)
rf_optimized_predictions = rf.predict(x_test)

In [ ]:
rf_optimized_scores = get_scores('RandomForest Optimized', y_test, rf_optimized_predictions)
rf_optimized_scores

{'model': 'RandomForest Optimized',
 'r2': 0.22190723610748586,
 'rmse': 1.8618457512601094}

In [ ]:
scores = save_scores(rf_optimized_scores, scores)
scores

,model,r2,rmse
0,XGBoost Regressor,0.098200,2.004395e+00
1,xgbr,-0.594543,2.004395e+00
2,XGBoost Regressor,-0.594543,2.004395e+00
3,Lightgbm,0.238616,1.841746e+00
4,LinearRegression,-0.000641,1.347647e+09
5,RandomForest,0.221907,1.861846e+00
6,RandomForest Optimized,0.221907,1.861846e+00


## **5. Evaluación y Selección del Modelo**


---


In [ ]:
# results_df = results_df.T
# colors = ['#0077b6', '#CDDBF3','#9370DB', '#DDA0DD']
# results_df.plot(kind='bar', figsize=(12, 6), colormap='viridis', rot=0, color=colors)
# plt.title('Comparación de Métricas por Modelo')
# plt.xlabel('Métricas')
# plt.ylabel('Puntuación')
# plt.legend(title='Modelos')
# plt.tight_layout()
# plt.show()

In [ ]:
def visualiza_resultados():
    pass

In [ ]:
visualiza_resultados()